In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd 
import pickle
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
# 默认情况下，matplotlib不支持中文显示，我们需要进行一下设置。
# 设置字体为黑体，以支持中文显示。
rcParams["font.family"] = "SimHei"
# 设置在中文字体时，能够正常的显示负号（-）。
rcParams["axes.unicode_minus"] = False

In [3]:
df = pd.read_excel("./data/blogs.xlsx",names=['columnid','content'])
# df.info()
# df.describe()
stop_words = ['\n','','。','，','|','【','】',':',' ','...','/','.','_','+','=','[',']','-']
# 分词处理
df.content = df.content.map(
    lambda aa : ' '.join([ x for x in " ".join(jieba.cut(str(aa))).split(' ') if x not in stop_words]))
# 写入
df.to_excel('./data/blogs_participled.xlsx')
tt = ' '.join([ x for x in df.content])
cco = set(tt.split(' ')) # 转集合去重
# 保存词汇列表
with open('./data/blogswordslist.pickle',mode='wb') as f1:
     pickle.dump(cco,f1)

In [4]:
# 加载词汇列表
with open('./data/blogswordslist.pickle',mode='rb') as f2:
     a_list = pickle.load(f2)
vocabulary = list(a_list)
# tfidf向量
stop_words = list(['\n','','。','，','|','【','】',':',' ','...','/','.','_','+','=','[',']','-'])

vectorizer = TfidfVectorizer(stop_words=stop_words,vocabulary=vocabulary)
aav = vectorizer.fit_transform(df.content.fillna(axis=0,method='ffill')) # 有缺失值用上一个填充
y_np = np.array(df['columnid'],dtype=float)
X_train, X_test, y_train, y_test = train_test_split(aav, y_np, test_size=0.3, random_state=0)
max_iter = 25 # 训练数据的最大次数
clfs = {
    '随机梯度下降(SGD)': SGDClassifier(max_iter=max_iter, tol=1e-3),
    '感知机(Perceptron)': Perceptron(max_iter=max_iter,tol=1e-3),
    '朴素贝叶斯分类器(MultinomialNB)': MultinomialNB(alpha=0.01),
    'PA被动感知算法(Passive-Aggressive)': PassiveAggressiveClassifier(max_iter=max_iter,tol=1e-3),
}
for cls_name, clf in clfs.items():
#     clf = clf.partial_fit(X_train,y_train,classes=[1,2,3,4,5,6,7,8,9,10,11]) # 增量学习
    clf = clf.fit(X_train,y_train)
    scores = cross_val_score(clf,X_test,y_test,cv=5,scoring='accuracy')
    print("%s Accuracy: %0.3f (+/- %0.3f)" % (cls_name,scores.mean(), scores.std() * 2))

随机梯度下降(SGD) Accuracy: 0.340 (+/- 0.036)
感知机(Perceptron) Accuracy: 0.314 (+/- 0.036)
朴素贝叶斯分类器(MultinomialNB) Accuracy: 0.347 (+/- 0.040)
PA被动感知算法(Passive-Aggressive) Accuracy: 0.338 (+/- 0.035)


In [5]:
cls_name = 'PA被动感知算法(Passive-Aggressive)'
clfp = PassiveAggressiveClassifier(max_iter=20,tol=1e-3)
clfp = clfp.fit(X_train,y_train)
scores = cross_val_score(clf,X_test,y_test,cv=5,scoring='accuracy')
print("%s Accuracy: %0.3f (+/- %0.3f)" % (cls_name,scores.mean(), scores.std() * 2))

PA被动感知算法(Passive-Aggressive) Accuracy: 0.341 (+/- 0.026)


In [6]:
# 保存模型
with open('./model/clfpa.model',mode='wb') as f1:
     pickle.dump(clfp,f1)
# 加载模型
with open('./model/clfpa.model',mode='rb') as f2:
     clfpa = pickle.load(f2)

In [7]:
ttii = '''惯性导航一般集成在GPS设备中 '''
type(tio)
len(ttii)
aa = []
aa.append(' '.join([ x for x in " ".join(jieba.cut(str(ttii))).split(' ') if x not in stop_words]))
aa
ace = vectorizer.fit_transform(aa)
clfpa.predict(ace)

['惯性导航 一般 集成 在 GPS 设备 中 都 是 由 供应商 集成 那 在 这里 有 什么 讨论 的 必要 呢 要 知道 在 车辆 行驶 中 我们 可以 拿到 GPS 的 yawrate 和 speed 信号 而且 车辆 本身 还有 一套 传感器 获取 yawrate 和 speed 又 因为 航迹 推算 是 自动 驾驶 很 重要 的 一部分 所以 理解 惯性导航 的 工作 原理 能 很 好 地 帮助 我们 做 基于 车身 的 航迹 推算 1 惯性导航 目前 GNSS IMU 构成 的 组合 导航系统']

array([6.])

array([4.])